In [ ]:
import open3d as o3d
import numpy as np
# 3/10

In [8]:
import numpy as np
import sys
sys.path.append(r"D:\基台的三维形变\AITeeth_deformation_src\袖口")

from utils import tooth_normal_utils, draw_utils, mesh_utils

# 使用三角形面积作为权值，每个三角形的法向量乘以权值求平均得到模型朝向,并且可以根据前一次的结果进行迭代
def find_normal_by_triangles_area_bayes_iteration(mesh, point, point_normal, iteration_num, start_num=1, show=False):
    if start_num <= iteration_num:
        above_plane_mesh, below_plane_mesh = tooth_normal_utils.find_above_plane_vertices_and_triangles(mesh, point,
                                                                                                        point_normal)
        above_average_normal = tooth_normal_utils.find_normal_by_triangles(above_plane_mesh)
        below_average_normal = tooth_normal_utils.find_normal_by_triangles(below_plane_mesh)
        average_normal = (above_average_normal + below_average_normal * 0.7) / 2
        return find_normal_by_triangles_area_bayes_iteration(mesh, point, average_normal, iteration_num, start_num + 1)
    else:
        if show:
            arrow = draw_utils.draw_line(origin=mesh_utils.get_mesh_centroid(mesh), direction=point_normal)
            draw_utils.draw_geometries(arrow, mesh)
        return point_normal


# 使用平分点的办法求出法向量
def find_tooth_normal_by_vertices_divide(mesh, iteration_num, top_weight=1, low_weight=0.1, show=False):
    vertices = mesh_utils.get_mesh_vertices(mesh)  # 获取顶点坐标
    normals = mesh_utils.get_mesh_vertices_normals(mesh)  # 获取顶点法向量
    norms = np.linalg.norm(normals, axis=1, keepdims=True)
    normals = normals / norms

    # 2. 计算质心和平均法向量
    centroid = vertices.mean(axis=0)  # 计算质心
    average_normal = normals.mean(axis=0)  # 计算平均法向量
    average_normal = average_normal / np.linalg.norm(average_normal)  # 归一化法向量

    normal = tooth_normal_utils.find_normal_by_vertices_divide_bayes_iteration(vertices, normals, average_normal, centroid, iteration_num,
                                                    top_weight, low_weight)
    if show:
        arrow = draw_utils.draw_line(origin=centroid, direction=normal)
        draw_utils.draw_geometries(arrow, mesh)
    return mesh, normal

In [ ]:
import os
import pymeshlab as ml
import open3d as o3d

def repair_and_visualize_mesh(input_path, output_path, depth=10):
    # 创建 MeshLab 实例
    ms = ml.MeshSet()

    # 加载网格
    ms.load_new_mesh(input_path)

    # 使用 MeshLab 的 Screened Poisson 表面重建功能
    ms.apply_filter("generate_surface_reconstruction_screened_poisson", depth=depth)

    # 保存修复后的网格
    ms.save_current_mesh(output_path)

    # 读取修复后的网格
    mesh_repaired = o3d.io.read_triangle_mesh(output_path)
    mesh_repaired.compute_vertex_normals()
    mesh_repaired.paint_uniform_color([0.1, 0.7, 0.3])

    # 可视化结果
    #o3d.visualization.draw_geometries([mesh_repaired])

def process_teeth_folder(input_folder, output_folder, depth=10):
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 遍历输入文件夹中的所有 .ply 文件
    for filename in os.listdir(input_folder):
        if filename.endswith(".ply"):
            # 构造输入文件的完整路径
            input_path = os.path.join(input_folder, filename)
            
            # 构造输出文件的完整路径
            output_filename = f"repaired_{filename}"
            output_path = os.path.join(output_folder, output_filename)

            # 处理并保存网格
            print(f"Processing {filename}...")
            repair_and_visualize_mesh(input_path, output_path, depth=depth)
            print(f"Saved repaired mesh to {output_path}")

# 示例调用
input_folder = r"D:\wzpdata\teeth2"  # 输入文件夹路径
output_folder = r"D:\wzpdata\teeth2\repaired"  # 输出文件夹路径
#process_teeth_folder(input_folder, output_folder, depth=10)

In [34]:
import os
import pymeshlab as ml
import open3d as o3d
arrows = []
# 遍历输入文件夹中的所有 .ply 文件
temp_folder = r"D:\wzpdata\teeth3\source2"
for filename in os.listdir(temp_folder):
    if filename.endswith(".ply"):
        output_path = os.path.join(temp_folder, filename)
        mesh_teeth = o3d.io.read_triangle_mesh(output_path)
        mesh_teeth.compute_vertex_normals()                    
        mesh_teeth.paint_uniform_color([0.1, 0.7, 0.3])
        # 提取mesh的点集
        vertices_teeth = np.asarray(mesh_teeth.vertices)

        #o3d.visualization.draw_geometries([mesh_teeth])
        # 提取顶点法向量
        normals = np.asarray(mesh_teeth.vertex_normals)
        # 计算法向量的平均值
        average_normal = np.mean(normals, axis=0)
        average_normal /= np.linalg.norm(average_normal)  # 归一化
        print("平均法向量（朝向）：", average_normal)
        _,average_normal=find_tooth_normal_by_vertices_divide(mesh_teeth, 2, 1, 0.1, True)
        direction_vector = average_normal

        center = np.mean(vertices_teeth, axis=0)
        start_point = center
        arrow_length = 10

        # 创建箭头
        arrow = o3d.geometry.TriangleMesh.create_arrow(
            cylinder_radius=0.02,  # 圆柱半径
            cone_radius=0.04,      # 圆锥半径
            cylinder_height=arrow_length * 0.8,  # 圆柱部分长度
            cone_height=arrow_length * 0.2,      # 圆锥部分长度
        )

        # 将箭头移动到起点
        arrow.translate(start_point)

        # 计算旋转矩阵
        z_axis = np.array([0, 0, 1])  # 默认箭头方向
        rotation_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(
            np.cross(z_axis, direction_vector)  # 计算旋转轴
        )
        arrow.rotate(rotation_matrix, center=start_point)  # 旋转箭头

        # 设置箭头颜色
        arrow.paint_uniform_color([1, 0, 0])  # 红色
        arrows.append(arrow)

        # 可视化箭头
        o3d.visualization.draw_geometries([arrow,mesh_teeth])

平均法向量（朝向）： [-0.13178675 -0.06164154  0.98935968]


In [35]:

file_path = r"D:\wzpdata\teeth3\teeth.obj"
mesh = o3d.io.read_triangle_mesh(file_path)
mesh.compute_vertex_normals()
mesh.paint_uniform_color([0.1, 0.7, 0.3])  # 设置网格颜色

# 提取网格的点集
vertices = np.asarray(mesh.vertices)

# 创建坐标框架
axis_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(size=20.0, origin=np.array([0, 0, 0]))

# 显示网格、箭头和坐标框架
o3d.visualization.draw_geometries(arrows + [mesh, axis_pcd])